In [64]:
import altair as alt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Final Report

Create an electronic report in English with a maximum of 2000 words (excluding citations) using Jupyter. The report should include the posed question, conducted analysis, and derived conclusion. Only one team member needs to submit this report. It is not required to include all tasks completed by every group member in their individual assignments; tailor the final report to the collective group's work. Make sure to reach a consensus among all team members on the final content of the report. If needed, consult your TA and Instructor for further guidance.

You must submit 2 files:

- an .html file (File -> Download As -> HTML)
- an .ipynb file. This file must be fully reproducible. It must run completely from top to bottom without any additional files.

## How Many Hours Would a Player Contribute Given Their “Age”?

## Introduction

Many studies conduct research about the effect of video games on the youth, and whether we should be concerned regarding how often young kids interact with them. Rehbein and Baier’s (2013) study highlights the issue of video games becoming addictive during youth, discussing the problematic use of video games and specific risk factors, and how these may correlate to gaining a gaming addiction. However, Witt et al. (2011) discuss if personality and self-esteem have an effect on video games and overall internet use. While they find there is a relationship, they also find that youth increase internet usage as they develop but decrease the amount of time they spend playing video games. Both studies shine light on the question of youth and video games, providing a basis as to our research question: how many hours does a player contribute to a video game given their age? 

The Pacific Laboratory for Artificial Intelligence (PLAI) at the University of British Columbia has begun a project to examine the relationship between video games like Minecraft, and the power of artificial intelligence. We can utilize its data regarding the age of players and how many hours they contribute to the game, to discover if we can predict whether players of a certain age will play more or less. The dataset used to answer our question is the players dataset, which involves information about the players’ characteristics. The dataset includes the following information such as a player’s experience level (“veteran” as the most experienced with Minecraft and “beginner” as the least), whether a player is subscribed, a player’s encrypted email, the total amount of hours played, the player’s nickname, the player’s gender, and the player’s age. A model can thus be made using the age and playing hours data to examine if a relationship is present and thus, if we can predict a player’s play time from how old they are.

## Methods and Results

### Exploratory Data Analysis

The data was loaded by retrieving the data on the github repository, and relavent columns are selected.

In [85]:
players_df = pd.read_csv('https://raw.githubusercontent.com/Caden-Z/DSCI100-Term-Project/refs/heads/main/data/players.csv')
players_df = players_df[['age', 'played_hours']]
players_df.head()

,age,played_hours
0,9,30.3
1,17,3.8
2,17,0.0
3,21,0.7
4,21,0.1


The dataframe is inspected for any missing values, result showed that there is no null values in the given observations.

In [86]:
missing_players = players_df.isnull().sum()
print("Missing values in key columns:")
print(missing_players)

Missing values in key columns:
age             0
played_hours    0
dtype: int64


The relationship between the relavent variables are visualized and inspected via graphs.

In [119]:
played_hours_chart = alt.Chart(players_df).mark_bar().encode(
    alt.X('played_hours:Q', bin=alt.Bin(maxbins=20), title='Played Time (hours)'),
    alt.Y('count()', title='Frequency')
).properties(
    title='Distribution of Played Hours',
    width=400,
    height=300
)

age_vs_playtime = alt.Chart(players_df).mark_circle(size=60, opacity=0.5).encode(
    alt.X('age:Q', title='Age (years)'),
    alt.Y('played_hours:Q', title='Play Time (hours)'),
    tooltip=['age', 'played_hours']
).properties(
    title='Scatter Plot of Age vs. Played Hours',
    width=400,
    height=300
)

played_hours_chart & age_vs_playtime

alt.VConcatChart(...)

The predictor and response columns were selected and stored in variables X and y for both the training set and testing set.

In addition, training and testing data split was made at the start to avoid model training with the test data that is used to evaluate the model’s performance after training.

In [120]:
train_df, test_df = train_test_split(players_df, test_size=0.25, random_state=42)

X_train = train_df[["age"]]
y_train = train_df["played_hours"]

X_test = test_df[["age"]]
y_test = test_df["played_hours"]

X_train.head()

,age
162,19
42,24
66,22
135,21
78,22


In [121]:
X_test.head()

,age
139,20
113,17
16,17
75,21
154,19


### Linear Regression Model

The linear regression model was created and fitted with the training data. 

Although no preprocessor was needed for this model because:
- There is no N/A observations, so no imputer is needed to synthesize the missing values
- There is only 1 predictor variable, thus no scaling is needed to standardize the unit between variables to remove the impact of different scales of different variables

The preprocessor is still used for good practice and in case more predictors are needed in the future.

In [122]:
lr_preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')), 
            ('scaler', StandardScaler())                 
        ]), ['age'])                                     
    ]
)

lr_pipeline = Pipeline(steps=[
    ('preprocessor', lr_preprocessor),
    ('regressor', LinearRegression())  
])

lr_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age'])])),
                ('regressor', LinearRegression())])

The linear regression model was evaluated using both RMSE and RMSPE.

In [123]:
lr_train_predictions = lr_pipeline.predict(X_train)
lr_test_predictions = lr_pipeline.predict(X_test)

lr_train_rmse = root_mean_squared_error(y_train, lr_train_predictions)
lr_test_rmspe = root_mean_squared_error(y_test, lr_test_predictions)

print("Linear Regression Training RMSE:", lr_train_rmse)
print("Linear Regression Test RMSPE:", lr_test_rmspe)

Linear Regression Training RMSE: 32.339101959393936
Linear Regression Test RMSPE: 8.00126786800662


Data frames were prepared with predicted values to be graphed

In [124]:
lr_players_predictions_train = train_df.assign(predicted=lr_train_predictions, set="Train")
lr_players_predictions_test = test_df.assign(predicted=lr_test_predictions, set="Test")

lr_combined_data = pd.concat([lr_players_predictions_train, lr_players_predictions_test], ignore_index=True)

# display only 5 columns
lr_combined_data.sample(n=5, random_state=3)

,age,played_hours,predicted,set
34,17,0.0,8.113149,Train
25,26,0.0,6.553422,Train
14,19,0.1,7.766543,Train
178,23,0.0,7.073331,Test
121,24,0.0,6.900028,Train


A scatter representing predicted vs. actual values were plotted to visaluze the performance of the model.

In [125]:
actual_vs_predicted = alt.Chart(lr_combined_data).mark_circle(size=60, opacity=0.7).encode(
    x=alt.X('played_hours', title='Actual Values'),
    y=alt.Y('predicted', title='Predicted Values'),
    color=alt.Color('set', scale=alt.Scale(domain=['Train', 'Test'], range=['blue', 'orange']), title='Dataset'),
    tooltip=['played_hours', 'predicted', 'set']
).properties(
    title='Actual vs Predicted Values for Training and Testing Sets',
    width=600,
    height=400
)

actual_vs_predicted

alt.Chart(...)

In addition to the actual vs. predicted graph, a graph representing the linear line produced by the linear regression model is overlayed with the orinal data graph to better visualize the model's performance. 

In [126]:
lr_chart = age_vs_playtime + alt.Chart(lr_combined_data).mark_line(color="orange").encode(
    x="age",
    y="predicted"
)

lr_chart

alt.LayerChart(...)

It can be seen that there are prevalent outliers in this data set that impacts the performance of the linear regression model, as they do not represent the amount of hours played in others of similar age. To accommodate, the outliers were removed by only analyzing and training using players who have played under 10 hours and are under the age of 60.

In [142]:
train_df, test_df = train_test_split(players_df[(players_df["played_hours"] < 10) & (players_df["age"] < 60)], test_size=0.25, random_state=42)

X_train = train_df[["age"]]
y_train = train_df["played_hours"]

X_test = test_df[["age"]]
y_test = test_df["played_hours"]
lr_pipeline.fit(X_train, y_train)

lr_train_predictions = lr_pipeline.predict(X_train)
lr_test_predictions = lr_pipeline.predict(X_test)

lr_train_rmse = root_mean_squared_error(y_train, lr_train_predictions)
lr_test_rmspe = root_mean_squared_error(y_test, lr_test_predictions)

print("Linear Regression Training RMSE:", lr_train_rmse)
print("Linear Regression Test RMSPE:", lr_test_rmspe)

Linear Regression Training RMSE: 1.0676593236098022
Linear Regression Test RMSPE: 0.710615929075977


In [143]:
lr_players_predictions_train = train_df.assign(predicted=lr_train_predictions, set="Train")
lr_players_predictions_test = test_df.assign(predicted=lr_test_predictions, set="Test")

lr_combined_data = pd.concat([lr_players_predictions_train, lr_players_predictions_test], ignore_index=True)

# display only 5 columns
lr_combined_data.sample(n=5, random_state=3)

lr_chart = alt.Chart(lr_combined_data).mark_circle(size=60, opacity=0.5).encode(
    alt.X('age:Q', title='Age (years)').scale(zero=False),
    alt.Y('played_hours:Q', title='Play Time (hours)'),
    tooltip=['age', 'played_hours']
).properties(
    title='Scatter Plot of Age vs. Played Hours',
    width=400,
    height=300
) + alt.Chart(lr_combined_data).mark_line(color="orange").encode(
    x="age",
    y="predicted"
)

lr_chart

alt.LayerChart(...)

### KNeighbors Regression Model

In [14]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  
    ('scaler', StandardScaler())                 
])

In [15]:
knn_preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, ['age'])  
    ]
)
knn_preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['age'])])

In [16]:
knn_pipeline = Pipeline(steps=[
    ('preprocessor', knn_preprocessor),
    ('regressor', KNeighborsRegressor(n_neighbors=5)) 
])
knn_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age'])])),
                ('regressor', KNeighborsRegressor())])

In [17]:
knn_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age'])])),
                ('regressor', KNeighborsRegressor())])

In [18]:
knn_train_predictions = knn_pipeline.predict(X_train)
knn_test_predictions = knn_pipeline.predict(X_test)

In [19]:
knn_train_rmse = root_mean_squared_error(y_train, knn_train_predictions)
knn_test_rmspe = root_mean_squared_error(y_test, knn_test_predictions)

In [20]:
print("KNN Regressor Training RMSE:", knn_train_rmse)
print("KNN Regressor Test RMSPE:", knn_test_rmspe)

KNN Regressor Training RMSE: 31.194567368907897
KNN Regressor Test RMSPE: 15.687777407905811


In [21]:
train_data = pd.DataFrame({'Actual': y_train, 'Predicted': knn_train_predictions, 'Set': 'Train'})
test_data = pd.DataFrame({'Actual': y_test, 'Predicted': knn_test_predictions, 'Set': 'Test'})
knn_combined_data = pd.concat([train_data, test_data], ignore_index=True)

display(knn_combined_data.head())

,Actual,Predicted,Set
0,0.0,1.64,Train
1,0.1,0.18,Train
2,0.0,43.70,Train
3,0.1,0.22,Train
4,0.1,1.64,Train


## References